In [ ]:
import pyperclip

from misc_util.logutils import setup_logging
from misc_util.pretty_print import Markdown, display

from derive_conceptualspace.pipeline import SnakeContext, load_envfiles
from derive_conceptualspace.util.result_analysis_tools import getfiles_allconfigs, highlight_nonzero_max, get_best_conf, df_to_latex
from derive_conceptualspace.cli.args_from_filename import get_filename, print_envvars
from derive_conceptualspace.evaluate.shallow_trees import classify_shallowtree_multi
from derive_conceptualspace.util.desc_object import DescriptionList
from derive_conceptualspace.pipeline import cluster_loader

## Find best-performing parameter-combination

In [ ]:
getfiles_allconfigs("clusters")

In [ ]:
setup_logging()
load_envfiles("placetypes")
conf, perf = get_best_conf("Geonames", verbose=True, balance_classes=True, one_vs_rest=True, dt_depth=1, test_percentage_crossval=0.33)

In [ ]:
print_envvars(get_filename(conf, get_dependencies=False, doprint=False))

In [ ]:
ctx = SnakeContext.loader_context(config=conf, silent=False)
descriptions, clusters, embedding = ctx.load("pp_descriptions", "clusters", "embedding", 
                          loaders=dict(pp_descriptions=DescriptionList.from_json, clusters=cluster_loader, embedding=lambda **args: args["embedding"].embedding_))

In [ ]:
res = classify_shallowtree_multi(clusters, embedding, descriptions, ctx.obj["dataset_class"], verbose=False)

In [ ]:
SHOW_CROSSVAL_NUM = 5

styles = [{'selector': 'th', 'props': [('vertical-align','top'),('text-align','left')]}] #('border-style', 'solid')  #see https://stackoverflow.com/a/55904239/5122790
styler = lambda df: df.style.apply(highlight_nonzero_max, axis=0).format(precision=3).set_table_styles(styles)
    
for taxonomy in set(res.index.get_level_values(0)):
    display(Markdown(f"## {taxonomy}"))
    df = res.xs(SHOW_CROSSVAL_NUM, level=1).xs(taxonomy).unstack(level=[1])
    display(styler(df))
    pyperclip.copy(df_to_latex(df, styler, rotate=False, caption="This algorithm on Placetypes"))